# **Data Cleaning and Processing**

In [1]:
#Libraries
import pandas as pd
import numpy as np
import rasterio
import os

## **1. Processing the Bird Data**

The bird data is in .csv format.

In [2]:
df = pd.read_csv('AVONET1_BirdLife.csv')
print(df.head().to_string())

   Sequence               Species1       Family1           Order1       Avibase.ID1  Total.individuals  Female  Male  Unknown  Complete.measures  Beak.Length_Culmen  Beak.Length_Nares  Beak.Width  Beak.Depth  Tarsus.Length  Wing.Length  Kipps.Distance  Secondary1  Hand-Wing.Index  Tail.Length   Mass Mass.Source Mass.Refs.Other Inference Traits.inferred Reference.species    Habitat  Habitat.Density  Migration Trophic.Level Trophic.Niche Primary.Lifestyle  Min.Latitude  Max.Latitude  Centroid.Latitude  Centroid.Longitude   Range.Size
0    3103.0  Accipiter albogularis  Accipitridae  Accipitriformes  AVIBASE-BBB59880                  5       2     0        3                  4                27.7               17.8        10.6        14.7           62.0        235.2            81.8       159.5             33.9        169.0  248.8     Dunning             NaN        NO             NaN               NaN     Forest                1        2.0     Carnivore     Vertivore       Insessorial     

To select the relevant columns and remove entries with missing data points:

In [3]:
cols_to_keep = [
    'Species1', 'Family1', 'Order1',

    'Beak.Length_Culmen', 'Beak.Width', 'Beak.Depth',
    'Tarsus.Length', 'Wing.Length', 'Hand-Wing.Index',
    'Tail.Length', 'Mass',
    
    'Habitat', 'Trophic.Level', 'Trophic.Niche', 'Primary.Lifestyle', 'Migration',

    'Range.Size', 'Min.Latitude', 'Max.Latitude',
    'Centroid.Longitude', 'Centroid.Latitude'
    
]

selected_cols = [c for c in cols_to_keep if c in df.columns]
df_clean = df[selected_cols].copy()
df_clean = df_clean.dropna()

print(df_clean.head().to_string())

                Species1       Family1           Order1  Beak.Length_Culmen  Beak.Width  Beak.Depth  Tarsus.Length  Wing.Length  Hand-Wing.Index  Tail.Length   Mass    Habitat Trophic.Level Trophic.Niche Primary.Lifestyle  Migration   Range.Size  Min.Latitude  Max.Latitude  Centroid.Longitude  Centroid.Latitude
0  Accipiter albogularis  Accipitridae  Accipitriformes                27.7        10.6        14.7           62.0        235.2             33.9        169.0  248.8     Forest     Carnivore     Vertivore       Insessorial        2.0     37461.21        -11.73         -4.02              158.49              -8.15
1       Accipiter badius  Accipitridae  Accipitriformes                20.6         8.8        11.6           43.0        186.7             32.9        140.6  131.2  Shrubland     Carnivore     Vertivore       Insessorial        3.0  22374973.00        -29.47         46.39               44.98               8.23
2      Accipiter bicolor  Accipitridae  Accipitriformes      

For the "Habitat" feature, there are 11 possible categories.

Coastal, Marine, Riverine, Wetland, Forest, Woodland, Shrubland, Grassland, Desert, Rock, Human Modified

To simplify the categories:

In [4]:
conditions = [
    # Aquatic/Marine
    df_clean["Habitat"].isin(['Coastal', 'Marine', 'Riverine', 'Wetland']),
    
    # Closed/Forest
    df_clean["Habitat"].isin(['Forest', 'Woodland', 'Shrubland']),
    
    # Open/Terrestrial
    df_clean["Habitat"].isin(['Grassland', 'Desert', 'Rock', 'Human Modified'])
]

choices = ['Aquatic', 'Closed', 'Open']

df_clean["Habitat"] = np.select(conditions, choices, default=None)

print(df_clean.head().to_string())

                Species1       Family1           Order1  Beak.Length_Culmen  Beak.Width  Beak.Depth  Tarsus.Length  Wing.Length  Hand-Wing.Index  Tail.Length   Mass Habitat Trophic.Level Trophic.Niche Primary.Lifestyle  Migration   Range.Size  Min.Latitude  Max.Latitude  Centroid.Longitude  Centroid.Latitude
0  Accipiter albogularis  Accipitridae  Accipitriformes                27.7        10.6        14.7           62.0        235.2             33.9        169.0  248.8  Closed     Carnivore     Vertivore       Insessorial        2.0     37461.21        -11.73         -4.02              158.49              -8.15
1       Accipiter badius  Accipitridae  Accipitriformes                20.6         8.8        11.6           43.0        186.7             32.9        140.6  131.2  Closed     Carnivore     Vertivore       Insessorial        3.0  22374973.00        -29.47         46.39               44.98               8.23
2      Accipiter bicolor  Accipitridae  Accipitriformes               

For the "Migration" feature, there are 3 possible categories.

1 = Sedentary
2 = Partial
3 = Migratory

To explicitly name the categories:

In [5]:
migration_map = {
    1.0: 'Sedentary',
    2.0: 'Partial',
    3.0: 'Migratory'
}

df_clean['Migration'] = df_clean['Migration'].map(migration_map)

print(df_clean.head().to_string())

                Species1       Family1           Order1  Beak.Length_Culmen  Beak.Width  Beak.Depth  Tarsus.Length  Wing.Length  Hand-Wing.Index  Tail.Length   Mass Habitat Trophic.Level Trophic.Niche Primary.Lifestyle  Migration   Range.Size  Min.Latitude  Max.Latitude  Centroid.Longitude  Centroid.Latitude
0  Accipiter albogularis  Accipitridae  Accipitriformes                27.7        10.6        14.7           62.0        235.2             33.9        169.0  248.8  Closed     Carnivore     Vertivore       Insessorial    Partial     37461.21        -11.73         -4.02              158.49              -8.15
1       Accipiter badius  Accipitridae  Accipitriformes                20.6         8.8        11.6           43.0        186.7             32.9        140.6  131.2  Closed     Carnivore     Vertivore       Insessorial  Migratory  22374973.00        -29.47         46.39               44.98               8.23
2      Accipiter bicolor  Accipitridae  Accipitriformes               

## **2. Adding the Climate Data**

wc2.1_10m_bio is a compilation of 19 .tif files. These files are raster images where each pixel on the grid corresponds to a specific coordinate and the corresponsing numerical climate value. 

For this project, relevant climatic variables are:

* bio_1: Average temperature
* bio_4: Temperature seasonality (standard deviation × 100)
* bio_5: Maximum temperature
* bio_6: Minimum temperature
* bio_12: Annual precipitation

Based on centroid longtitude and latitude of each record, these climatic variables are added at the end of the data as new columns.

In [6]:
SELECTED_BIOS = [1, 4, 5, 6, 12]

coords = [(x, y) for x, y in zip(df_clean['Centroid.Longitude'], df_clean['Centroid.Latitude'])]

for i in SELECTED_BIOS:
    tif_path = os.path.join('wc2.1_10m_bio', f"wc2.1_10m_bio_{i}.tif")
    if os.path.exists(tif_path):
        with rasterio.open(tif_path) as src:
            vals = []
            for x in src.sample(coords, masked=True):
                val = x[0]
                if np.ma.is_masked(val):
                    vals.append(np.nan)
                else:
                    vals.append(val)
            
            df_clean[f'bio_{i}'] = vals

df_clean.dropna(inplace=True)

df_clean.to_csv('AVONETplusClim.csv', index=False)
print(df_clean.head().to_string())

                Species1       Family1           Order1  Beak.Length_Culmen  Beak.Width  Beak.Depth  Tarsus.Length  Wing.Length  Hand-Wing.Index  Tail.Length   Mass Habitat Trophic.Level Trophic.Niche Primary.Lifestyle  Migration   Range.Size  Min.Latitude  Max.Latitude  Centroid.Longitude  Centroid.Latitude      bio_1        bio_4      bio_5      bio_6  bio_12
1       Accipiter badius  Accipitridae  Accipitriformes                20.6         8.8        11.6           43.0        186.7             32.9        140.6  131.2  Closed     Carnivore     Vertivore       Insessorial  Migratory  22374973.00        -29.47         46.39               44.98               8.23  24.756603   144.596039  32.647999  15.486000   316.0
2      Accipiter bicolor  Accipitridae  Accipitriformes                25.0         8.6        12.7           58.1        229.6             24.6        186.3  287.5  Closed     Carnivore     Vertivore        Generalist    Partial  14309701.27        -55.72         23.73  